In [ ]:
# pip install langchain faiss-cpu cohere
# pip install langchain-community langchain-cohere python-dotenv

import os
from dotenv import load_dotenv

load_dotenv()
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

In [26]:
knowledge_base_text = """
LangChain is an open-source framework designed to help developers build applications powered by large language models.
It provides tools for chaining prompts, managing memory, and integrating with external data sources.
Python is a versatile programming language used widely in data science, AI, web development, and more.
Generative AI refers to a class of artificial intelligence models capable of generating text, images, or other data types based on input prompts.
"""

with open("knowledge_base.txt", "w") as file:
    file.write(knowledge_base_text)

In [27]:
from langchain.document_loaders import TextLoader

loader = TextLoader('knowledge_base.txt')
documents = loader.load()

In [28]:
from langchain_cohere import CohereEmbeddings
from langchain.vectorstores import FAISS

embedding = CohereEmbeddings(cohere_api_key = COHERE_API_KEY, model = "embed-english-v3.0")
vectorstore = FAISS.from_documents(documents, embedding)

In [29]:
from langchain_cohere import ChatCohere

chat_model = ChatCohere(
    model = "command-r-plus",
    cohere_api_key = COHERE_API_KEY
)

In [30]:
from langchain_core.messages import HumanMessage

'''
def chatbot():
    print("Message LangChain AI Chatbot. Type 'exit' to end the chat.")
    while True:
        query = input("You: ")
        if query.lower() == 'exit':
            print("Goodbye!")
            break
        
        # relevant context from the knowledge base
        docs = vectorstore.similarity_search(query, k = 1)
        context = docs[0].page_content if docs else "No relevant information found."
        
        prompt = f"Based on the following context, answer the user's query:\n\nContext: {context}\n\nQuery: {query}"
        user_message = HumanMessage(content = prompt)
        
        response = chat_model.invoke([user_message])
        print(f"AI: {response.content}")

chatbot()
'''

'\ndef chatbot():\n    print("Message LangChain AI Chatbot. Type \'exit\' to end the chat.")\n    while True:\n        query = input("You: ")\n        if query.lower() == \'exit\':\n            print("Goodbye!")\n            break\n        \n        # relevant context from the knowledge base\n        docs = vectorstore.similarity_search(query, k = 1)\n        context = docs[0].page_content if docs else "No relevant information found."\n        \n        prompt = f"Based on the following context, answer the user\'s query:\n\nContext: {context}\n\nQuery: {query}"\n        user_message = HumanMessage(content = prompt)\n        \n        response = chat_model.invoke([user_message])\n        print(f"AI: {response.content}")\n\nchatbot()\n'

In [31]:
from langchain.memory import ConversationBufferMemory

'''
memory = ConversationBufferMemory()

def chatbot_with_memory():
    print("Message LangChain AI Chatbot with Memory. Type 'exit' to end the chat.")
    while True:
        query = input("You: ")
        if query.lower() == 'exit':
            print("Goodbye!")
            break
        
        docs = vectorstore.similarity_search(query, k = 1)
        context = docs[0].page_content if docs else "No relevant information found."
        
        # adding context and query to memory
        memory.chat_memory.add_user_message(query)
        memory.chat_memory.add_ai_message(f"Context: {context}")
        
        prompt = f"Conversation History:\n{memory.buffer}\n\nBased on the following context, answer the user's query:\n\nContext: {context}\n\nQuery: {query}"
        user_message = HumanMessage(content = prompt)
        response = chat_model.invoke([user_message])
        
        # storing response
        memory.chat_memory.add_ai_message(response.content)
        
        print(f"AI: {response.content}")

chatbot_with_memory()
'''

'\nmemory = ConversationBufferMemory()\n\ndef chatbot_with_memory():\n    print("Message LangChain AI Chatbot with Memory. Type \'exit\' to end the chat.")\n    while True:\n        query = input("You: ")\n        if query.lower() == \'exit\':\n            print("Goodbye!")\n            break\n        \n        docs = vectorstore.similarity_search(query, k = 1)\n        context = docs[0].page_content if docs else "No relevant information found."\n        \n        # adding context and query to memory\n        memory.chat_memory.add_user_message(query)\n        memory.chat_memory.add_ai_message(f"Context: {context}")\n        \n        prompt = f"Conversation History:\n{memory.buffer}\n\nBased on the following context, answer the user\'s query:\n\nContext: {context}\n\nQuery: {query}"\n        user_message = HumanMessage(content = prompt)\n        response = chat_model.invoke([user_message])\n        \n        # storing response\n        memory.chat_memory.add_ai_message(response.conten

In [32]:
def test_non_memory_chatbot():
    queries = [
        "I used LangChain back in 2022 for a personal project at my university",
        "When have I used LangChain before?"
    ]
    print("\n--- Testing Non-Memory Chatbot ---")
    for query in queries:
        docs = vectorstore.similarity_search(query, k = 1)
        context = docs[0].page_content if docs else "No relevant information found."
        
        prompt = f"Based on the following context, answer the user's query:\n\nContext: {context}\n\nQuery: {query}"
        user_message = HumanMessage(content = prompt)
        response = chat_model.invoke([user_message])
        print(f"You: {query}\nAI: {response.content}\n")

def test_memory_chatbot():
    queries = [
        "I used LangChain back in 2022 for a personal project at my university",
        "When have I used LangChain before?"
    ]
    print("\n--- Testing Memory Chatbot ---")

    memory = ConversationBufferMemory()

    for query in queries:
        docs = vectorstore.similarity_search(query, k = 1)
        context = docs[0].page_content if docs else "No relevant information found."
                
        memory.chat_memory.add_user_message(query)
        memory.chat_memory.add_ai_message(f"Context: {context}")
        
        prompt = f"Conversation History:\n{memory.buffer}\n\nBased on the following context, answer the user's query:\n\nContext: {context}\n\nQuery: {query}"
        user_message = HumanMessage(content = prompt)
        response = chat_model.invoke([user_message])
        
        memory.chat_memory.add_ai_message(response.content)
        print(f"You: {query}\nAI: {response.content}\n")

test_non_memory_chatbot()
test_memory_chatbot()


--- Testing Non-Memory Chatbot ---
You: I used LangChain back in 2022 for a personal project at my university
AI: Yes, LangChain has been available as an open-source framework since 2021. So, it is very possible that you used LangChain for your personal project at the university in 2022.

You: When have I used LangChain before?
AI: You have not mentioned any specific instances of using LangChain in your provided context. However, based on the information given, it can be inferred that you might have used LangChain if you were a developer working on building applications powered by large language models and required a framework to help with prompt chaining, memory management, and data integration.


--- Testing Memory Chatbot ---
You: I used LangChain back in 2022 for a personal project at my university
AI: It's great to hear that you've had experience with LangChain! It's an excellent tool for developers looking to harness the power of large language models. Was your personal project 